# Dimension Reduction via Sinusoidal Projection

### EECS 16B: Designing Information Devices and Systems II

# Motivation

In this python exercise we will demonstrate how projections can be beneficial in classification. We will use the simple model where we have noisy observations from (known) sinusoidal signals, and try to classify observations to the true sinusoids. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.cluster
from mpl_toolkits.mplot3d import Axes3D

## Creating data

In our example, we have known sinusoids of frequency $1,3,5$ Hz. In our demo, we will suppose there are three true sinusoids. For each of the frequencies, we observe $100$ noisy data vectors (so $300$ total), each consisting of a set of $1000$ sampled points. The observations are corrupted by noise scaled by $\sigma$. You may play around with these values to see what happens.

These noisy samples are stored in the variable `sinusoids`. This variable will have `num_sinusoids * num_data` rows and `sample` columns.

This means that the row vectors of `sinusoids` will be all of the samples of a sine wave at some frequency, with added noise.

In [ ]:
num_sinusoids = 3 # number of true sinusoids
num_data = 100 # number of data vectors for each sinusoid
Fs = 1000 # sample rate
sample = 1*Fs # number of samples
sigma = 1.0 # noise scale

def generate_sinusoids(num_sinusoids, num_data, Fs, sample, sigma):
    """
    Generates noisy sinusoidal data for us. Hint: use np.random.normal() function with loc = 0.0 and scale = sigma for noise.
    inputs:
      num_sinusoids: number of true sinusoids
      num_data: number of data vectors for each sinusoid
      Fs: sample rate
      sample: number of samples
      sigma: # noise scale
    returns:
      presorted: a list of num_sinusoids lists, with each sublist having num_data amount of noisy sinusoidal vectors
    """
    presorted = []
    for k in range(num_sinusoids):
        f = 2*k+1 # Frequency of Signal
        data = []
        for i in range(num_data):
            x = np.arange(sample)
            vec = np.sin(2 * np.pi * f * x / Fs) + np.random.normal(loc=0.0, scale=sigma, size=[sample,])
            data.append(vec)
        presorted.append(data)
    return presorted

presorted = generate_sinusoids(num_sinusoids, num_data, Fs, sample, sigma)
sinusoids = np.concatenate([np.array(i) for i in presorted])

We also generate true sinusoid values.

In [ ]:
true_list = generate_sinusoids(num_sinusoids, 1, Fs, sample, 0.0)
true_sinusoid = np.concatenate([np.array(i) for i in true_list])

## Plotting our observations

Let's visualize the data we generated. In the first plot, we plot a picture of $10$ randomly selected sinusoidal shapes. In the following plots, we plot the noisy waveforms for each of the true sinusoids, and plot the average waveforms.

In [ ]:
# randomly choose 10 waveforms to plot
indices = np.random.choice(len(sinusoids), size = 10, replace = False)
x_lims = [i/Fs for i in range(0, sample)]
for waveforms in sinusoids[indices]:
    plt.plot(x_lims, waveforms)
plt.xlim((0,sample/Fs))
plt.title('10 random sinusoidal shapes')
plt.figure()

# plot the 3 sinusoidal shapes based on the presorted data
for i, waveforms in enumerate(presorted):
    plt.title('Waveforms of sinusoidal with frequency '+str(2*i+1)+' Hz')
    for data in waveforms:
        plt.plot(x_lims, data)
        plt.xlim((0,sample/Fs))
    plt.figure()
    plt.title('Average waveform of sinusoidal with frequency '+str(2*i+1)+' Hz')
    plt.plot(x_lims, np.mean(waveforms, axis=0))
    plt.xlim((0,sample/Fs))
    plt.plot(x_lims, true_sinusoid[i], label = 'True Sinusoid')
    plt.legend()
    plt.figure()
    

## Verifying Orthogonality
We have our three true sinusoids stored in the `true_sinusoid` variable. Let us verify that these three sinusoids are orthogonal. 

In [ ]:
# Verify that the 1Hz is orthongonal to the 3Hz Sine-Wave
inner_prod_1_3 = np.inner(true_sinusoid[0], true_sinusoid[1])
print(f'Inner product of 1-Hz with 3-Hz sine waves = {inner_prod_1_3}')

# Verify that the 3Hz is orthongonal to the 5Hz Sine-Wave
inner_prod_3_5 = np.inner(true_sinusoid[1], true_sinusoid[2])
print(f'Inner product of 3-Hz with 5-Hz sine waves = {inner_prod_3_5}')

# Verify that the 1Hz is orthongonal to the 5Hz Sine-Wave
inner_prod_1_5 = np.inner(true_sinusoid[0], true_sinusoid[2])
print(f'Inner product of 1-Hz with 5-Hz sine waves = {inner_prod_1_5}')

We notice that each inner product is zero (close to zero due to floating point error). That is, the rows of `true_sinusoid` are all orthogonal to one another.

There are many resources explaining why this is true. We will ask you to blindly take this as a fact for now, this should become clear after learning about the DFT. 

## Projecting data onto the sinusoidal waves

One way to classify data is to project data onto lower dimensions in hope of having a nice separation in a lower dimension. In this part, we observe what happens when we project the data points onto the linear space spanned by the known sinusoids. 

What does projection mean in this case? In our example, we have $3$ vectors corresponding to the three true frequencies. Let's call them $\vec{v_1}$, $\vec{v_2}$ and $\vec{v_3}$ (these are the rows of `true_sinusoid`). For some noisy vector $\vec{y}$, we would want to find the best representation of $\vec{y}$ in terms of linear combinations of $\vec{v_1}$, $\vec{v_2}$ and $\vec{v_3}$. In other words, we would like to find the coefficients $a_1$, $a_2$ and $a_3$ such that
$$
    \vec{y} \approx a_1\vec{v_1} + a_2\vec{v_2} + a_3\vec{v_3}.
$$

The coefficient $a_1$ can be found by finding the inner product of $y$ and $v_1$ and normalizing:
$$
    a_1 = \frac{\langle y, \vec{v_1}\rangle}{\langle \vec{v_1}, \vec{v_1}\rangle}.
$$

Similarly, we can find $a_2$ and $a_3$ in this fashion.

Next, we define a function to calculate the projections.

In [ ]:
def projection(true_sinusoid, sinusoids):
    """
    Projects noisy sinusoids onto the true sinusoids
    inputs:
      true_sinusoid (3 X 1000 numpy array): vectors corresponding to the true_sinusoid
      sinusoids (300 X 1000 numpy array): the noisy observations
    returns:
      presorted_projected (300 X 3 numpy array): projections of the noisy observations onto the true sinusoids
    """
    presorted_projected = np.zeros((sinusoids.shape[0], true_sinusoid.shape[0]))
    rows, cols = presorted_projected.shape
    for r in range(rows):
        for c in range(cols):
            noisy_sig = sinusoids[r]
            true_sine = true_sinusoid[c,:]
            # TODO: Your code here
            # store the projection of coefficient of noisy_sig onto true_sine in `proj`
            proj = ?
            # ---------------------------------------------------------------------------------
            presorted_projected[r,c] = proj
    return presorted_projected

presorted_projected = projection(true_sinusoid, sinusoids)

Let's plot out our projections onto a 3D plot to see how they behave.

In [ ]:
def plot_3D(data, view_from_top=False):
    """ 
    Takes list of arrays (x, y, z) coordinate triples and plots 3D map
    input:
      data: num_sinusoids*num_data*num_sinusoids size array of data
      view_from_top: boolean function determining if we plot 3D plot from top
    """
    fig=plt.figure(figsize=(10,7))
    ax = fig.add_subplot(111, projection='3d')
    colors = ['#0000ff', '#00ff00', '#ff0000']
    lables = ['1-Hz Sinusoids Projections', '3-Hz Sinusoids Projections', '5-Hz Sinusoids Projections']
    for dat, color, lab in zip(data, colors, lables):
        Axes3D.scatter(ax, *dat.T, s=1, c=color, alpha=1.0, label=lab)
    if view_from_top:
        ax.view_init(elev=90.,azim=0) # Move perspective to view from top
    ax.set_xlabel('a1')
    ax.set_ylabel('a2')
    ax.set_zlabel('a3')
    ax.legend(loc='lower right')

def projection_plot(num_sinusoids, presorted_projected):
    """ 
    Plots 3D projection using data from presorted_projected and the plot_3D function
    input:
      num_sinusoids: number of true frequencies
      presorted_projected: array of projected values
    """
    plot_data = []
    for j in range(num_sinusoids):
        plot_data.append(presorted_projected[j*num_data:(j+1)*num_data])

    plot_3D(np.array(plot_data), False)
    plt.title('Presorted data projected to 3 sinusoidal components')

    plot_3D(np.array(plot_data), True)
    plt.title('Top View: Presorted data projected to 3 sinusoidal components')

projection_plot(num_sinusoids, presorted_projected)

## Classifying noisy data

From the previous part, we see that after projection, each projected noisy data has one index that is much larger  than the other two indices. We can use this as our classification algorithm. 

In [ ]:
def classify(projections):
    """
    Classifies projections based on largest indices; for example, if there are 3 true frequencies, return 0, 1 or 2 if the largest indice is in the 0th, 1st or 2nd position respectively
    input: 
      projections (300 X 3 numpy array): array of projected values
    returns:
      ind (300 numpy array): array of max indices in each row. 
    """
    ind = np.zeros(projections.shape[0])
    for i in range(len(ind)):
        a1_a2_a3 = projections[i]
        # TODO: Your code here
        # Hint: Think about using the function np.argmax
        ind[i] = ?
        # ------------------------------------------------------------------------------------------
    return ind

classifications = classify(presorted_projected)

def num_errors(num_sinusoids, classifications):
    """
    Returns number of errors between our classification and the true classes
    input:
      num_sinusoids: number of true frequencies
      classifications: vector of classifications
    output:
      error: number of misclassifications
    """
    # Remember our classification is in terms of indices. To compare this classification with the true frequencies, we create a representation of true class in terms of indices
    true_class = []
    for j in range(num_sinusoids):
        true_class = true_class + [j]*num_data
    ans = sum(i != j for i, j in zip(classifications, true_class))
    return ans

print("Number of errors in classification:", num_errors(num_sinusoids, classifications), "/", num_sinusoids*num_data)    

## Conclusion

Knowing the right dimension to project onto can be useful for many classification tasks. In our setting, we were already given the true sinusoidal values. However, in a practical setting this information is often not readily available to us. In the latter parts of the class we will learn other methods that can help us tackle this issue.

If you played with the noise values, you will also notice that with large enough noise, this method will not work. This is intuitive since we start out with sinusoidal signals with magnitude at most 1. If we add large enough noise, then it will be very hard to distinguish true signal from noise. 